In [1]:
import requests
import pandas as pd
import domojupyter as domo

url_token = "https://accounts.snapchat.com/login/oauth2/access_token"
payload_token = (
    "grant_type=refresh_token&refresh_token=XXXXXXX&client_id=XXXXXXX"
)
headers_token = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cookie': 'sc-wcid=XXXXXXXXX'
}

response_token = requests.post(url_token, headers=headers_token, data=payload_token)

if response_token.status_code == 200:
    access_token = response_token.json().get("access_token", "")

    urls = {
        "Main": "https://api.snapkit.com/v1/stories/studio/revenue/creator/XXXXX/aggregate?start_date=2024-01-01&end_date=2026-01-01",
        "Spanish": "https://api.snapkit.com/v1/stories/studio/revenue/creator/XXXXX/aggregate?start_date=2024-01-01&end_date=2026-01-01",
        "French": "https://api.snapkit.com/v1/stories/studio/revenue/creator/XXXXX/aggregate?start_date=2024-01-01&end_date=2026-01-01",
        "Arabic": "https://api.snapkit.com/v1/stories/studio/revenue/creator/XXXXX/aggregate?start_date=2024-01-01&end_date=2026-01-01"
    }
    
    headers_data = {
        'accept': 'application/json',
        'authorization': f'Bearer {access_token}'
    }
    
    timeseries_data_list = []

    for name, url in urls.items():
        response = requests.get(url, headers=headers_data)
        if response.status_code == 200:
            data = response.json()

            if 'timeseries_metrics' in data['metadata']:
                timeseries_metrics = data['metadata']['timeseries_metrics']
                df_timeseries = pd.json_normalize(timeseries_metrics)
                
                df_timeseries['date'] = pd.to_datetime(df_timeseries['date']).dt.tz_localize(None)
                
                df_timeseries['Snap.Channel'] = name
                timeseries_data_list.append(df_timeseries)
            else:
                print(f"No 'timeseries_metrics' data found for {name}")
        else:
            print(f"Failed to retrieve data for {name}: {response.text}")
            
    if timeseries_data_list:
        combined_df = pd.concat(timeseries_data_list, ignore_index=True)
        
        combined_df['_BATCH_LAST_RUN_'] = pd.Timestamp.now().date()

        domo.write_dataframe(combined_df, 'Snapchat | Combined Aggregated Revenue | Output')
        
        print("Success")
    else:
        print("No data retrieved.")

A newer version of domojupyter is available. Update domojupyter with the following command:

	conda install -c https://domo-conda-prod.s3.amazonaws.com/domo domojupyter=1.1.32
